# Fast Curvelet Transform Demo

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lzepedanunez/fast_curvelet_transform/blob/main/notebooks/curvelet_demo.ipynb)

This notebook demonstrates the usage of the Fast Discrete Curvelet Transform (FDCT) in Python. We cover:
1. **Forward and Inverse Transform**: Verifying the identity.
2. **Scale-by-Scale Reconstruction**: Visualizing how different frequency bands contribute to the image.
3. **Curvelet Denoising**: Using soft-thresholding on curvelet coefficients to remove noise.

## 0. Installation and Setup
If you are running this on Google Colab, you need to install the package first.

In [ ]:
# Install the package (Uncomment if running on Colab)
# !pip install git+https://github.com/lzepedanunez/fast_curvelet_transform.git

import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from fast_curvelet_transform.curvelet import fdct, ifdct, CurveletOptions

print("Setup complete.")

## 1. Load Image and Setup Options

In [ ]:
# Load image
image = data.camera().astype(np.float64) / 255.0
m, n = image.shape

# Setup options
options = CurveletOptions(
    is_real=True,
    m=m,
    n=n,
    nbscales=5,
    nbangles_coarse=16,
    finest="wavelets",
    dtype=np.float64
)

print(f"Image loaded: {m}x{n}")

## 2. Forward and Inverse Transform
We verify that the reconstructed image matches the original.

In [ ]:
print("Computing transforms...")
coeffs = fdct(image, options)
reconstructed = ifdct(coeffs, options)

error = np.linalg.norm(image - reconstructed) / np.linalg.norm(image)
print(f"Relative Reconstruction Error: {error:.4e}")

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(image, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(122)
plt.imshow(reconstructed, cmap='gray')
plt.title("Reconstructed")
plt.axis('off')
plt.show()

## 3. Scale-by-Scale Reconstruction
We visualize the components of the image at each frequency scale.

In [ ]:
nbscales = options.nbscales
fig, axes = plt.subplots(1, nbscales, figsize=(20, 5))

for j in range(nbscales):
    # Create a copy with only one scale active
    coeffs_scale = []
    for scale_idx in range(nbscales):
        if scale_idx == j:
            coeffs_scale.append(coeffs[scale_idx])
        else:
            coeffs_scale.append([np.zeros_like(a) for a in coeffs[scale_idx]])
    
    rec_scale = ifdct(coeffs_scale, options)
    axes[j].imshow(rec_scale, cmap='gray')
    axes[j].set_title(f"Scale {j}")
    axes[j].axis('off')

plt.tight_layout()
plt.show()

## 4. Denoising via Soft-Thresholding
We add noise to the image and use the curvelet transform to denoise it.

In [ ]:
def soft_threshold(x, threshold):
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

# Add noise
sigma = 0.05
noisy_image = image + sigma * np.random.randn(m, n)

# Forward transform of noisy image
noisy_coeffs = fdct(noisy_image, options)

# Apply thresholding
threshold = 0.5 * sigma
denoised_coeffs = []
for scale in noisy_coeffs:
    denoised_scale = [soft_threshold(a, threshold) for a in scale]
    denoised_coeffs.append(denoised_scale)

# Inverse transform
denoised_image = ifdct(denoised_coeffs, options)

# Calculate PSNR
psnr_noisy = 10 * np.log10(1 / np.mean((image - noisy_image)**2))
psnr_denoised = 10 * np.log10(1 / np.mean((image - denoised_image)**2))

print(f"Noisy PSNR: {psnr_noisy:.2f} dB")
print(f"Denoised PSNR: {psnr_denoised:.2f} dB")

# Visualization
plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.imshow(image, cmap='gray')
plt.title("Original")
plt.axis('off')

plt.subplot(132)
plt.imshow(noisy_image, cmap='gray')
plt.title(f"Noisy (PSNR={psnr_noisy:.1f}dB)")
plt.axis('off')

plt.subplot(133)
plt.imshow(denoised_image, cmap='gray')
plt.title(f"Denoised (PSNR={psnr_denoised:.1f}dB)")
plt.axis('off')

plt.tight_layout()
plt.show()